In [ ]:
import os
import datetime
import pytesseract
from PIL import Image

In [ ]:
images_path = "C:\\1_projects\\138_fedgis2021\\OCR\\images\\"
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [ ]:
arcpy.env.workspace = images_path
current_time = datetime.datetime.now().strftime("%B_%d_%Y_%I_%M_%S%p")
arcpy.CreateFileGDB_management(images_path, current_time + '.gdb')
SR = arcpy.SpatialReference(4326)
new_point = arcpy.CreateFeatureclass_management(images_path + current_time + '.gdb', "Images", 'POINT', spatial_reference=SR)
fc = new_point[0]
arcpy.AddField_management(fc, "Name", "TEXT", "", "", 100, "Name")
arcpy.AddField_management(fc, "City", "TEXT", "", "", 100, "City")
arcpy.AddField_management(fc, "File", "TEXT", "", "", 100, "File")

for filename in os.listdir(images_path):
    if filename.endswith(".JPG") or filename.endswith(".PNG") or filename.endswith(".jpg") or filename.endswith(".png"):
        text = pytesseract.image_to_string(Image.open(images_path + filename))
        split_all = text.split("\n")
        name = split_all[0]
        city = split_all[1]
        coords = split_all[2].split(" Lng: ")
        lat = coords[0].replace("Lat: ", "")
        lng = coords[1]
        with arcpy.da.InsertCursor(fc, ['SHAPE@', 'Name', "City", "File"]) as cursor:
            coordinates = arcpy.Point(lng,lat)
            cursor.insertRow((coordinates, name, city, filename))
        print(name + " added.")
        
print("\nExport complete.")